In [4]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys
sys.path.insert(1, '/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/util_funcs/')
from pandas_colFuncs import whichBisSig_NAHS, B_ID, ancestor_below_B,D0_decay_type

In [129]:
save_data = True
save_preprocessedDataframe = True
tmp_data = False
take_subset = False
subset_size = 100000

delete_top_numFSP_evts=True

In [6]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "NAHSA_Gmodes_fixedD0modes/"
root_subdir = "NAHS_allEvts_twoSubs_fixedD0run/"  

root_path = nfs_path + "NAHS/onlineDataProd/" + root_subdir 
#root_path = "/afs/desy.de/user/a/axelheim/private/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/"

In [62]:
merged = ""
sub = "sub1/"

In [63]:
fileHc = uproot.open(root_path + sub + merged + "allHc.root:variables")

In [64]:
df_Hc = fileHc.arrays(library="pd")

In [65]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename=root_path + sub  + merged + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_twoSubs_fixedD0run/sub1/gammas.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_twoSubs_fixedD0run/sub1/electrons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_twoSubs_fixedD0run/sub1/pions.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_twoSubs_fixedD0run/sub1/kaons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_twoSubs_fixedD0run/sub1/muons.root:variables


In [66]:
df_FSPs = pd.concat(dfs)

In [67]:
# delete FSPs for which no Y4S file entry was found
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [68]:
df_FSPs.keys()

Index(['__experiment__', '__run__', '__event__', '__candidate__',
       '__ncandidates__', '__weight__', 'genMothPDG_0', 'genMothPDG_1',
       'genMothPDG_2', 'genMothPDG_3', 'genMothPDG_4', 'genMothPDG_5',
       'genMothPDG_6', 'genMothPDG_7', 'genMothPDG_8', 'genMothPDG_9',
       'genMotherID_0', 'genMotherID_1', 'genMotherID_2', 'genMotherID_3',
       'genMotherID_4', 'genMotherID_5', 'genMotherID_6', 'genMotherID_7',
       'genMotherID_8', 'genMotherID_9', 'mcMother0_uniqParID',
       'mcMother1_uniqParID', 'mcMother2_uniqParID', 'mcMother3_uniqParID',
       'mcMother4_uniqParID', 'mcMother5_uniqParID', 'mcMother6_uniqParID',
       'mcMother7_uniqParID', 'mcMother8_uniqParID', 'mcMother9_uniqParID',
       'cmpx', 'cmpy', 'cmpz', 'cmE', 'isSignal', 'uniqueParticleIdentifier',
       'mcErrors', 'mcPDG', 'genMotherID', 'genMotherP', 'genMotherPDG',
       'charge', 'dr', 'dz', 'clusterReg', 'clusterE9E21', 'M', 'PDG',
       'genParticleID', 'px', 'py', 'pz', 'pt', 'p', 'E'

In [69]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])
print(df_Hc["__event__"].nunique())

26999933
1619955
1530407


## delete evts where Hc mother and Bsig overlap

In [70]:
df_Hc['Bsig_uniqParID'] = df_Hc.apply(whichBisSig_NAHS, axis=1)

In [71]:
df_Hc['Bsig_uniqParID'].value_counts()

83886081.0    851908
83886082.0    768047
Name: Bsig_uniqParID, dtype: int64

In [72]:
print(df_Hc.shape[0])

1619955


In [73]:
# these events have Hc as a Bsig daughter, which is of course wrong
df_Hc[df_Hc['Bsig_uniqParID'] == df_Hc['mcMother0_uniqParID']].shape[0]

762364

In [74]:
df_Hc = df_Hc[df_Hc['Bsig_uniqParID'] != df_Hc['mcMother0_uniqParID']]

In [75]:
print(df_Hc.shape[0])

857591


In [76]:
print(df_FSPs.shape[0])
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]
print(df_FSPs.shape[0])

26999933
14911631


## check the subset of (B-sig side) D*-> D0 (-> kpi, kpipi,...) pi  

In [77]:
df_Hc['D0_decay'] = df_Hc.apply(D0_decay_type, axis=1)

In [78]:
df_Hc['D0_decay'].value_counts()

notWanted     716972
Kpipi0         76544
Kpipipi        26380
Kpi            19122
Kpipipipi0     18573
Name: D0_decay, dtype: int64

In [79]:
# select only these D0 modes and D*+ -> D0 pi
#D0:kpi 
#D0:kpipi pi 
#D0:kpipi0     
#D0:kpipi pi pi0 

In [80]:
df_Hc = df_Hc[df_Hc['D0_decay'] != "notWanted"]

In [81]:
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [82]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])

2369118
140619


## add cols

In [83]:
# function to create col with the particles mother B's uniqueParticleIdentifier
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

In [84]:
df_FSPs['B_ID'].value_counts()

83886082    1021947
83886081    1018209
0            328962
Name: B_ID, dtype: int64

In [85]:
df_FSPs['PDG'].value_counts()

 22.0     1169307
 211.0     307111
-211.0     290732
 13.0      120850
-13.0      118953
 321.0     112144
-321.0     109864
-11.0       70605
 11.0       69552
Name: PDG, dtype: int64

In [86]:
df_Hc['PDG'].value_counts() ## must have 7 different particles

-411.0     32520
 411.0     32033
 413.0     22807
-413.0     22244
 421.0     12177
-421.0      8095
-431.0      2865
 431.0      2856
 423.0      1647
 443.0      1378
-423.0      1047
 4122.0      490
-4122.0      460
Name: PDG, dtype: int64

## add labels to FSPs

In [87]:
df_FSPs = pd.merge(df_FSPs,df_Hc[["__event__","Bsig_uniqParID"]], on="__event__")

In [88]:
def labels(s):
    label = -1
    if int(s['B_ID']) == 0:
        label = 0 # background, cause not related to MC Particles
    else: 
        B_sigID = s['Bsig_uniqParID']
        
        if int(s['B_ID']) == B_sigID:
            label = 2 # Bsig
        else:
            label = 1 #X
    return label

In [89]:
df_FSPs['label'] = df_FSPs.apply(labels, axis=1)

## check D*lnu

In [90]:
groupsHc = pd.DataFrame({'count' : df_Hc[df_Hc["Bsig_uniqParID"] == 83886082.0].groupby( 
    ["genUp4S_PDG_1_0","genUp4S_PDG_1_1","genUp4S_PDG_1_2"] ).size()}).reset_index()

groupsHc

,genUp4S_PDG_1_0,genUp4S_PDG_1_1,genUp4S_PDG_1_2,count
0,-413.0,-13.0,14.0,16683
1,-413.0,-11.0,12.0,16802
2,413.0,11.0,-12.0,16694
3,413.0,13.0,-14.0,16742


In [91]:
groupsHc = pd.DataFrame({'count' : df_Hc[df_Hc["Bsig_uniqParID"] == 83886081.0].groupby( 
    ["genUp4S_PDG_0_0","genUp4S_PDG_0_1","genUp4S_PDG_0_2"] ).size()}).reset_index()

groupsHc

,genUp4S_PDG_0_0,genUp4S_PDG_0_1,genUp4S_PDG_0_2,count
0,-413.0,-13.0,14.0,18237
1,-413.0,-11.0,12.0,18695
2,413.0,11.0,-12.0,18396
3,413.0,13.0,-14.0,18370


## take care of duplis in df_Hc, delete them, because with the prior, duplis here mean there must have been an error

In [92]:
groupsHc_uniqParID = pd.DataFrame({'count' : df_Hc.groupby( ["__event__"] ).size()}).reset_index()
groupsHc_uniqParID.value_counts('count')

count
1    135630
2      2424
3        40
4         4
5         1
dtype: int64

In [93]:
df_Hc[df_Hc['__event__'].isin(groupsHc_uniqParID[groupsHc_uniqParID["count"]>1]["__event__"])][["__event__",
    "uniqueParticleIdentifier","PDG",
    "genMothPDG_0",'Bsig_uniqParID',"mcMother0_uniqParID","px","E","x","genUp4S_PDG_1_0"]][:10]

,__event__,uniqueParticleIdentifier,PDG,genMothPDG_0,Bsig_uniqParID,mcMother0_uniqParID,px,E,x,genUp4S_PDG_1_0
310,5934768,100663296.0,421.0,-511.0,83886082.0,83886081.0,0.786486,2.378538,-0.038657,-413.0
311,5934768,100663296.0,421.0,-511.0,83886082.0,83886081.0,0.730800,2.131254,-0.038657,-413.0
1094,26602407,100663296.0,421.0,511.0,83886081.0,83886082.0,0.202732,2.086966,-0.040929,311.0
1161,40054826,100663296.0,421.0,511.0,83886081.0,83886082.0,0.415799,1.917817,-0.047296,421.0
1162,40054826,100663296.0,421.0,511.0,83886081.0,83886082.0,-0.691797,2.044667,0.000000,421.0
1320,26870211,100663296.0,421.0,-511.0,83886081.0,83886082.0,-0.448989,2.319895,-0.055442,-431.0
1442,6604366,100663296.0,421.0,511.0,83886082.0,83886081.0,-0.002371,1.918710,-0.047663,413.0
1547,497,100663296.0,-421.0,511.0,83886081.0,83886082.0,1.064082,2.131342,-0.060829,-421.0
1548,497,100663296.0,-421.0,511.0,83886081.0,83886082.0,1.051044,2.117427,-0.040485,-421.0
1647,34325774,100663296.0,421.0,-511.0,83886082.0,83886081.0,0.610207,2.139448,-0.045228,-413.0


In [94]:
df_Hc = df_Hc[df_Hc['__event__'].isin(groupsHc_uniqParID[groupsHc_uniqParID["count"]==1]["__event__"])]

In [95]:
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [96]:
groupsHc_uniqParID = pd.DataFrame({'count' : df_Hc.groupby( ["__event__"] ).size()}).reset_index()
groupsHc_uniqParID.value_counts('count')

count
1    135630
dtype: int64

## delete the H_c FSPs 

In [97]:
df_FSPs['ancestor_below_B_arrayID'] = df_FSPs.apply(ancestor_below_B, axis=1)

In [98]:
df_Hc["HcMother_uniqParID"] = df_Hc["mcMother0_uniqParID"]
df_Hc["Hc_genParticleID"] = df_Hc["genParticleID"]
df_Hc["Hc_MotherPDG"] = df_Hc["genMothPDG_0"]

In [99]:
df_FSPs = pd.merge(df_FSPs,df_Hc[["__event__","Bsig_uniqParID","HcMother_uniqParID","Hc_genParticleID","Hc_MotherPDG"]], on="__event__")

In [100]:
df_FSPs.shape[0]

2325807

In [101]:
# deletion of Hc FSPs, those who 
df_FSPs = df_FSPs[~((df_FSPs["Hc_genParticleID"] == df_FSPs["ancestor_below_B_arrayID"]) &  # originate from Hc arrayID(genParticleID)
                  (df_FSPs["HcMother_uniqParID"] == df_FSPs["B_ID"]))] # and are B-tag ancestors

In [102]:
df_FSPs.shape[0]

1752678

## delete delete_top_numFSP_evts

In [103]:
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
numEvtsBeforeCut = df_Hc.shape[0]

In [104]:
# print first to adjust percentile used
print("original max FSPs:",numberFSPsEvts["count"].max())
for percentile_lvl in [0.995,0.996,0.997,0.998,0.999]:
    print(percentile_lvl)
    percentile = numberFSPsEvts["count"].quantile(q=percentile_lvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    #TMP_df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    TMP_df_Hc = df_Hc[df_Hc['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
    print("would delete", round(100 - (TMP_df_Hc.shape[0]/df_Hc.shape[0])*100,2),"% of evts")
    print("new max FSPs:",cut_numberFSPsEvts["count"].max())

original max FSPs: 32
0.995
would delete 0.87 % of evts
new max FSPs: 22
0.996
would delete 0.44 % of evts
new max FSPs: 23
0.997
would delete 0.44 % of evts
new max FSPs: 23
0.998
would delete 0.23 % of evts
new max FSPs: 24
0.999
would delete 0.11 % of evts
new max FSPs: 25


In [105]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])


if delete_top_numFSP_evts:
    delete_percentileLvl = 0.997
    print("used delete_percentileLvl:",delete_percentileLvl)
    
    
    print(numberFSPsEvts["count"].describe())
    
    percentile = numberFSPsEvts["count"].quantile(q=delete_percentileLvl)
    cut_numberFSPsEvts = numberFSPsEvts[numberFSPsEvts["count"] < percentile]
    
    df_FSPs = df_FSPs[df_FSPs['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    df_Hc = df_Hc[df_Hc['__event__'].isin(cut_numberFSPsEvts["__event__"])]
    
print(df_FSPs.shape[0])
print(df_Hc.shape[0])
print("deleted", round(100 - (df_Hc.shape[0]/numEvtsBeforeCut)*100,4),"% of events")
numberFSPsEvts = pd.DataFrame({'count' : df_FSPs.groupby(["__event__"]).size()}).reset_index()
print(numberFSPsEvts["count"].describe())

1752678
135630
used delete_percentileLvl: 0.997
count    135629.000000
mean         12.922590
std           3.840367
min           1.000000
25%          10.000000
50%          13.000000
75%          15.000000
max          32.000000
Name: count, dtype: float64
1737736
135031
deleted 0.4416 % of events
count    135031.000000
mean         12.869163
std           3.762819
min           1.000000
25%          10.000000
50%          13.000000
75%          15.000000
max          23.000000
Name: count, dtype: float64


## check that each particles occurs only ones; count must be == 1 for all

In [106]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
groupsFSPs_uniqParID.value_counts('count')

count
1    1737736
dtype: int64

## impute the nan values of variables

In [107]:
## data preprocessing, dealing with NaN
imputelist = [["pionID",-1.],["kaonID",-1.],["electronID",-1.],["muonID",-1.],["protonID",-1.]
              ,["clusterReg",-1.],["clusterE9E21",-1.]
              ]
for impute in imputelist:
    column_name = impute[0]
    impute_val = impute[1]
    mask = df_FSPs[column_name].isna() == 1
    df_FSPs.loc[mask, column_name] = impute_val
    print(df_FSPs[df_FSPs[column_name].isna() == True].shape[0])

0
0
0
0
0
0
0


## last check if events match between dfs

In [108]:
df_Hc = df_Hc[df_Hc['__event__'].isin(df_FSPs["__event__"])]
df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Hc["__event__"])]

In [109]:
print(df_FSPs.shape[0])
print(df_Hc.shape[0])

1737736
135031


## merge x,y,z  vertex info of Hc to FSPs

In [110]:
#for coordinate in ["x","y","z"]:
#    print(df_Hc[coordinate].describe())    
#    print(df_Hc[f'{coordinate}_uncertainty'].describe())
#    print("\n")


In [111]:
df_FSPs = pd.merge(df_FSPs,df_Hc[["__event__","x","x_uncertainty","y","y_uncertainty","z","z_uncertainty"]], on="__event__")

## save the dfs as they will be used for data prod

In [112]:
df_FSPs_final = df_FSPs
df_Y4S_final = df_Hc

In [113]:
root_path

'/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/NAHS/onlineDataProd/NAHS_allEvts_twoSubs_fixedD0run/'

In [114]:
df_FSPs_final.to_csv(root_path + sub + "sub_df_FSPs_preProcessed_NAHS.csv")
df_Y4S_final.to_csv(root_path + sub + "sub_df_Y4S_preProcessed_NAHS.csv")

## load the df's and concat them

In [115]:
dfs0=[]
dfs1=[]

for sub in ["sub0/","sub1/"]:
    tmp_df_FSPs = pd.read_csv(root_path + sub + "sub_df_FSPs_preProcessed_NAHS.csv".format(subset_size))
    tmp_df_Y4S = pd.read_csv(root_path + sub + "sub_df_Y4S_preProcessed_NAHS.csv")
    
    dfs0.append(tmp_df_FSPs)
    dfs1.append(tmp_df_Y4S)

In [116]:
df_FSPs_final = pd.concat(dfs0)
df_Y4S_final = pd.concat(dfs1)

## delete evts which occur more than ones (ie in both subs; those with more than the earlier restricted number of particles per event (23 in case for Gmodes 2 subs data))

# NOTE: this step is probably unnecessary, only __event__ and __production__ are needed both to identify specific events 

In [117]:
df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

In [118]:
deleteEvts = df_FSPs_final[df_FSPs_final['numFSPs'] > 23]["__event__"]

In [119]:
df_Y4S_final = df_Y4S_final[~df_Y4S_final['__event__'].isin(deleteEvts)]

In [120]:
df_FSPs_final = df_FSPs_final[df_FSPs_final['__event__'].isin(df_Y4S_final["__event__"])]

## filter D* -> D0 pi

In [135]:
df_Y4S_final = df_Y4S_final[((df_Y4S_final["Bsig_uniqParID"] == 83886081.0)  &
            (df_Y4S_final["genUp4S_PDG_0_0_0"].abs() == 421.0)  & # 421=D0
            (df_Y4S_final["genUp4S_PDG_0_0_1"].abs() == 211.0)  & # 211=pion
            (df_Y4S_final["genUp4S_PDG_0_0_2"].isna() == True)   # no particle 
              |
        ((df_Y4S_final["Bsig_uniqParID"] == 83886082.0)  &
            (df_Y4S_final["genUp4S_PDG_1_0_0"].abs() == 421.0)  & # 421=D0
            (df_Y4S_final["genUp4S_PDG_1_0_1"].abs() == 211.0)  & # 211=pion
            (df_Y4S_final["genUp4S_PDG_1_0_2"].isna() == True)))] # no particle 

## save final result

In [136]:
df_FSPs_final.to_csv(root_path + "final_df_FSPs_preProcessed_NAHS.csv")
df_Y4S_final.to_csv(root_path + "final_df_Y4S_preProcessed_NAHS.csv")

## load final df's 

In [5]:
df_FSPs_final = pd.read_csv(root_path + "final_df_FSPs_preProcessed_NAHS.csv")
df_Y4S_final = pd.read_csv(root_path + "final_df_Y4S_preProcessed_NAHS.csv")

In [137]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()
numFSPs["count"].describe()

count    270740.000000
mean         12.869406
std           3.767360
min           1.000000
25%          10.000000
50%          13.000000
75%          15.000000
max          23.000000
Name: count, dtype: float64

## beginning of data prod

In [138]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')
print("num Events:",numFSPs.shape[0],'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 1
maxFSPs: 23 

num Events: 270740 



In [139]:
variable_combinations = [
    [[],"NAHSA_only4mom"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_allExtras"],
    [["charge","dr","clusterE9E21"],"NAHSA_chrg_dr_E9E21"],
    [[    "charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_M"],
    [["M",         "dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_charge"],
    [["M","charge",     "dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_dr"],
    [["M","charge","dr",     "clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_dz"],
    [["M","charge","dr","dz",             "clusterE9E21","pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_clReg"],
    [["M","charge","dr","dz","clusterReg",               "pionID","kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_E9E21"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21",         "kaonID","electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_pID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID",         "electronID","muonID","protonID",
     "x","y","z"],"NAHSA_no_kID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID",             "muonID","protonID",
     "x","y","z"],"NAHSA_no_eID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID"         ,"protonID",
     "x","y","z"],"NAHSA_no_muID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID"           ,
     "x","y","z"],"NAHSA_no_proID"],
    [["M","charge","dr","dz","clusterReg","clusterE9E21","pionID","kaonID","electronID","muonID","protonID"
                 ],"NAHSA_no_xyz"],
    [["charge","pionID","muonID","x","y","z"],"NAHSA_charge_mpIDs_xyz"],
    [["charge"],"NAHSA_charge"]
]

In [140]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-11-05 16:10:11.759815


In [141]:
minFSPs = 5 # minimum: Bsig: K pi pi lepton, X: 1 
for var_comb in variable_combinations:
    ablationSub = var_comb[1]
    data_dir = Path(nfs_path + "data/" + data_subdir + root_subdir + ablationSub + "/")    
    if save_data:
        data_dir.mkdir(parents=True, exist_ok=True)
    print("Will save data to:", data_dir,'is', save_data ,'\n')


    
    for num_FSPs_toData in range(minFSPs, maxFSPs+1):
        df_num_subset = df_FSPs_final.copy()
        df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]


        numEvents = df_num_subset.__event__.nunique()
        print("numEvents:",numEvents)
        print("num_FSPs_toData:",num_FSPs_toData)  
        if numEvents == 0:
            print("skipped because empty \n")
            continue

        if numEvents < 10:
            print("skipped because <10 events \n")
            continue

        num_features = 4 + len(var_comb[0])
        leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
        SA_target =  np.zeros((numEvents, num_FSPs_toData))
        global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)

        event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
        #print("len(event_list):",len(event_list))
        for i in range(numEvents):

            event_iter = event_list[i]

            global_tag_masterInfo = "evt" + str(event_iter)
            global_tag[i,-1] = global_tag_masterInfo
            #print("global_tag[i,-1]:",global_tag[i,-1])
            #print("i:",i,"event_iter:",event_iter)

            event_df = df_num_subset[df_num_subset.__event__ == event_iter]

            for j in range(num_FSPs_toData):
                #print("numParticle:",j)
                particle = event_df.iloc[j]

                #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
                leaves[i,j,0] = particle["px"]
                leaves[i,j,1] = particle["py"]
                leaves[i,j,2] = particle["pz"]
                leaves[i,j,3] = particle["E"]
                
                for k in range(len(var_comb[0])):
                    leav_idx = k + 4
                    leaves[i,j,leav_idx] = particle[var_comb[0][k]]
                    



                global_tag_Info = str((particle["mcPDG"])) 
                global_tag[i,j] = global_tag_Info

        
                SA_target[i,j] = particle["label"]

            del event_df


        # shuffle the data    
        for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
            perms = np.random.permutation(leaves.shape[1])

            leaves[idx,:] = leaves[idx,perms]
            SA_target[idx,:] = SA_target[idx,perms]
            global_tag[idx,0:-1] = global_tag[idx,perms]





        #print(global_tag)
        train_ratio = 0.8
        validation_ratio = 0.2
        #test_ratio = 0.1

        print("leaves.shape:",leaves.shape)
        print("SA_target.shape:",SA_target.shape)
        print("global_tag.shape:",global_tag.shape)


        print("leaves[0]:",leaves[0])
        print("SA_target[0]:",SA_target[0])
        print("global_tag[0]:",global_tag[0])

        x=leaves
        y=SA_target
        z=global_tag

        x_train, x_val, y_train, y_val, z_train, z_val = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
        #x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

        if save_data==True:
            np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
            np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
            np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

            np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
            np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
            np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

            #np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
            #np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
            #np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)


        print("")


        del df_num_subset


Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/NAHSA_Gmodes_fixedD0modes/NAHS_allEvts_twoSubs_fixedD0run/NAHSA_only4mom is True 

numEvents: 3
num_FSPs_toData: 1
skipped because <10 events 

numEvents: 41
num_FSPs_toData: 2
leaves.shape: (41, 2, 4)
SA_target.shape: (41, 2)
global_tag.shape: (41, 3)
leaves[0]: [[ 1.35836112  0.72657472  1.23995996  1.97751276]
 [ 0.78813308 -0.02116983  0.09064399  0.93463106]]
SA_target[0]: [1. 2.]
global_tag[0]: [b'-11.0' b'-321.0' b'evt3584248']

numEvents: 270
num_FSPs_toData: 3
leaves.shape: (270, 3, 4)
SA_target.shape: (270, 3)
global_tag.shape: (270, 4)
leaves[0]: [[-0.06010113  0.14264253  0.45463175  0.48025937]
 [ 0.51177484  0.30579427  0.63014138  0.87862487]
 [ 0.02052581  0.07534899  0.10790049  0.13319645]]
SA_target[0]: [2. 1. 0.]
global_tag[0]: [b'22.0' b'-211.0' b'nan' b'evt38139747']

numEvents: 1080
num_FSPs_toData: 4
leaves.shape: (1080, 4, 4)
SA_target.shape: (1080, 4)
global_tag.shape: (1080, 5

leaves.shape: (25810, 14, 4)
SA_target.shape: (25810, 14)
global_tag.shape: (25810, 15)
leaves[0]: [[ 0.84197807 -0.47932872 -0.01747457  0.97475748]
 [-0.06090977 -0.11904498  0.10101699  0.21809638]
 [-0.15441665  0.20521854 -0.05013439  0.29656787]
 [-0.2457924  -0.77243519  0.26540399  0.98550813]
 [ 0.40663555  0.13763399  0.0855593   0.45945171]
 [-0.11961485  0.27792966  0.15009807  0.33776033]
 [-0.27847731  0.90401131  0.23229662  0.97403685]
 [ 0.05694385  0.14645293  0.03781316  0.16161961]
 [ 0.02161388 -0.27397108  0.10354272  0.29368124]
 [ 0.03921726 -0.04116892 -0.02400384  0.06171756]
 [ 0.16851139  0.00760756  0.09309559  0.19266747]
 [-0.59693778  0.06485385  0.34646904  0.70715021]
 [ 0.29358113 -0.14885715  0.09701797  0.37046013]
 [ 0.99077868  0.10180441  0.24408615  1.02546811]]
SA_target[0]: [2. 2. 1. 2. 1. 1. 1. 1. 2. 0. 2. 2. 2. 1.]
global_tag[0]: [b'-13.0' b'-211.0' b'-211.0' b'321.0' b'-211.0' b'22.0' b'22.0' b'-211.0'
 b'-211.0' b'nan' b'-211.0' b'211.0' b


numEvents: 3382
num_FSPs_toData: 21
leaves.shape: (3382, 21, 4)
SA_target.shape: (3382, 21)
global_tag.shape: (3382, 22)
leaves[0]: [[-1.15076981e-01  2.18466282e-01  1.10995494e-01  3.04581877e-01]
 [-4.72936571e-01 -1.61684442e+00 -3.38964134e-01  1.71835730e+00]
 [ 5.68043860e-03 -2.59021968e-01  1.86901227e-01  3.19463168e-01]
 [ 6.01408407e-02 -4.32170220e-02  1.80850998e-01  1.95427007e-01]
 [ 3.74672972e-02 -3.62070054e-02  1.13087542e-01  1.24513203e-01]
 [-6.46156550e-04 -4.39727232e-02 -2.60017049e-02  5.10892020e-02]
 [ 1.22764446e-02  1.06787838e-01  6.18827492e-02  1.24031561e-01]
 [-6.84179291e-02 -7.42341876e-02  1.71099097e-01  1.98662096e-01]
 [-6.44813925e-02  3.22374731e-01 -3.45652960e-02  3.58828607e-01]
 [-7.99813420e-02 -8.12886581e-02 -7.76825473e-02  1.37983474e-01]
 [-1.55005022e-03  1.13332309e-01  2.46980458e-01  2.71746135e-01]
 [ 1.84058309e-01  5.27728796e-01  5.31061530e-01  7.70974548e-01]
 [ 2.79370964e-01  2.80862689e-01  4.44842577e-01  6.11798003e-


numEvents: 2845
num_FSPs_toData: 5
leaves.shape: (2845, 5, 18)
SA_target.shape: (2845, 5)
global_tag.shape: (2845, 6)
leaves[0]: [[ 8.99958089e-02 -1.06106244e-01 -1.02062888e-01  1.72553220e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.35546875e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00 -1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.42593533e-01 -2.38747329e-01 -4.27143931e-01  5.09691221e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   3.00000000e+00  9.16992188e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00 -1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-3.07131380e-01 -9.68562141e-02 -1.79846674e-01  3.94379930e-01
   1.39570385e-01  1.00000000e+00  1.19516608e-03  5.37646019e-03
   2.00000000e+00  9.23828125e-01  9.16423723e-01  2.59965062e-13
   5.46533861e-02  2.89228911e-02  8.77479810e-36  0.00000000e+00
   0.000


numEvents: 18645
num_FSPs_toData: 9
leaves.shape: (18645, 9, 18)
SA_target.shape: (18645, 9)
global_tag.shape: (18645, 10)
leaves[0]: [[-2.65031636e-01 -1.11164224e+00  3.99093926e-01  1.21850158e+00
   1.39570385e-01 -1.00000000e+00  1.95233678e-03  7.99208570e-02
   2.00000000e+00  1.00000000e+00  6.56852164e-01  3.41774303e-01
   1.92664094e-08  1.22610124e-08  1.37350179e-03 -5.20306341e-02
   2.31468417e-02  5.01925834e-02]
 [ 1.48050621e-01 -1.49941981e-01  2.79334813e-01  3.49899303e-01
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.97070312e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -5.20306341e-02
   2.31468417e-02  5.01925834e-02]
 [ 1.05508196e+00 -1.01096690e+00  1.38512170e+00  2.01340859e+00
   0.00000000e+00  0.00000000e+00  1.18686603e-09  2.23517418e-10
   2.00000000e+00  9.80468750e-01 -1.00000000e+00 -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00 -1.00000000e+00 -5.20306341e-02
   

...


numEvents: 11643
num_FSPs_toData: 18
leaves.shape: (11643, 18, 10)
SA_target.shape: (11643, 18)
global_tag.shape: (11643, 19)
leaves[0]: [[-5.08684479e-02  2.48666089e-02  1.60626285e-02  5.88553758e-02
   0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -4.70604710e-02
   1.34067014e-02  4.40919626e-04]
 [ 6.45235777e-02 -5.23308776e-02  1.07788764e-01  1.36089053e-01
   0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -4.70604710e-02
   1.34067014e-02  4.40919626e-04]
 [ 1.74113199e-01  1.11463726e-01 -5.69086783e-02  2.55847725e-01
  -1.00000000e+00  1.75162449e-01  1.90286638e-03 -4.70604710e-02
   1.34067014e-02  4.40919626e-04]
 [ 2.07081050e-01 -6.78471029e-01  4.97721523e-01  8.72981044e-01
  -1.00000000e+00  9.25275406e-03  9.90747245e-01 -4.70604710e-02
   1.34067014e-02  4.40919626e-04]
 [ 2.22667251e-02  6.48047626e-02  9.16064158e-02  1.14399299e-01
   0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -4.70604710e-02
   1.34067014e-02  4.40919626e-04]
 [ 3.13804448e-01  1.812037


numEvents: 3382
num_FSPs_toData: 21
leaves.shape: (3382, 21, 10)
SA_target.shape: (3382, 21)
global_tag.shape: (3382, 22)
leaves[0]: [[-6.44813925e-02  3.22374731e-01 -3.45652960e-02  3.58828607e-01
  -1.00000000e+00  9.99999862e-01  1.38440677e-07 -5.10687567e-02
   2.35734787e-02 -6.61315620e-02]
 [-1.19774602e-01  2.31382877e-01  2.91585922e-01  3.91032404e-01
   0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -5.10687567e-02
   2.35734787e-02 -6.61315620e-02]
 [ 3.74672972e-02 -3.62070054e-02  1.13087542e-01  1.24513203e-01
   0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -5.10687567e-02
   2.35734787e-02 -6.61315620e-02]
 [-7.12438300e-02 -6.50601983e-02  3.02811384e-01  3.17810080e-01
   0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -5.10687567e-02
   2.35734787e-02 -6.61315620e-02]
 [-6.46156550e-04 -4.39727232e-02 -2.60017049e-02  5.10892020e-02
   0.00000000e+00 -1.00000000e+00 -1.00000000e+00 -5.10687567e-02
   2.35734787e-02 -6.61315620e-02]
 [-1.55005022e-03  1.13332309e-


Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/NAHSA_Gmodes_fixedD0modes/NAHS_allEvts_twoSubs_fixedD0run/NAHSA_charge is True 

numEvents: 3
num_FSPs_toData: 1
skipped because <10 events 

numEvents: 41
num_FSPs_toData: 2
leaves.shape: (41, 2, 5)
SA_target.shape: (41, 2)
global_tag.shape: (41, 3)
leaves[0]: [[ 1.35836112  0.72657472  1.23995996  1.97751276  1.        ]
 [ 0.78813308 -0.02116983  0.09064399  0.93463106 -1.        ]]
SA_target[0]: [1. 2.]
global_tag[0]: [b'-11.0' b'-321.0' b'evt3584248']

numEvents: 270
num_FSPs_toData: 3
leaves.shape: (270, 3, 5)
SA_target.shape: (270, 3)
global_tag.shape: (270, 4)
leaves[0]: [[ 0.51177484  0.30579427  0.63014138  0.87862487 -1.        ]
 [-0.06010113  0.14264253  0.45463175  0.48025937  0.        ]
 [ 0.02052581  0.07534899  0.10790049  0.13319645  0.        ]]
SA_target[0]: [1. 2. 0.]
global_tag[0]: [b'-211.0' b'22.0' b'nan' b'evt38139747']

numEvents: 1080
num_FSPs_toData: 4
leaves.shape: (1080, 

leaves.shape: (27010, 13, 5)
SA_target.shape: (27010, 13)
global_tag.shape: (27010, 14)
leaves[0]: [[-0.01618002 -0.10223757  0.06339474  0.12138042  0.        ]
 [-0.30117601 -0.08275584  0.28913125  0.65182005  1.        ]
 [ 1.69937599  0.79175967 -0.03182442  1.87801454 -1.        ]
 [-0.95928693 -0.45345503  0.7602697   1.30532099  0.        ]
 [ 0.46631134  0.72483313  0.25450766  0.89866761  0.        ]
 [ 0.08953707  0.00481639  0.05675516  0.10611896  0.        ]
 [-0.03129036  0.0797062  -0.14550267  0.16882888  0.        ]
 [ 0.2776275  -0.22842607  0.22580674  0.42455205 -1.        ]
 [-0.38076156 -0.10995463  0.3469387   0.52672179  0.        ]
 [-0.06911616 -0.01208754  0.01307649  0.07137329  0.        ]
 [-0.00916055 -0.12192457  0.07251601  0.14215516  0.        ]
 [-0.06117035 -0.05394757 -0.06264355  0.10284146  0.        ]
 [-0.08442178  0.08756821 -0.04758492  0.13061222  0.        ]]
SA_target[0]: [0. 2. 2. 2. 1. 1. 1. 2. 2. 0. 0. 0. 1.]
global_tag[0]: [b'nan' b'2


numEvents: 8128
num_FSPs_toData: 19
leaves.shape: (8128, 19, 5)
SA_target.shape: (8128, 19)
global_tag.shape: (8128, 20)
leaves[0]: [[ 0.06368838  0.05031096  0.11938167  0.14435853  0.        ]
 [ 0.10686582  0.13271748  0.0828687   0.21694502 -1.        ]
 [ 0.13775636 -0.32929742 -0.16198757  0.41609311 -1.        ]
 [ 0.03808656 -0.08824143  0.13664602  0.16706068  0.        ]
 [ 0.04097902  0.11542553 -0.10795031  0.16326544  0.        ]
 [ 0.16827951  0.01976178  0.04154564  0.22341543 -1.        ]
 [-0.89683801 -0.0089689   0.61669046  1.08844199  0.        ]
 [ 0.35809138 -0.1916398   0.54915047  0.68302378  0.        ]
 [ 0.27711639  0.01858791  0.0382387   0.31317901  1.        ]
 [ 0.0839645  -0.02208122  0.01334361  0.08783889  0.        ]
 [ 1.59189296 -0.43690136  0.21609318  1.66484309 -1.        ]
 [ 0.38794184 -0.16685894  0.27634361  0.52362817  1.        ]
 [ 0.07171253 -0.30649304  0.23019131  0.38995988  0.        ]
 [-0.14718314 -0.05731917  0.11579816  0.1958509

In [ ]:
print("saving is done")
now = datetime.now()
print("time at end =", now)